In [1]:
import os
import glob
import netCDF4
import logging
import numpy as np
import cmocean
import emodnetchemistry
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
import calendar
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
plt.rcParams.update({'font.size': 16})
plt.rc('figure', facecolor='w')

In [2]:
figdir = "../figures/WOA2018/"
woadataurl = "https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa/oxygen/all/1.00/woa18_all_o00_01.nc"
divafile = "/data/EMODnet/Eutrophication/Products/Results-0.1/Water body dissolved oxygen concentration_background_test_it200.nc"
deltalon = 0.1
deltalat = 0.1
domain = [-40., 55., 24., 67.]
longrid = np.arange(domain[0], domain[1] + .0001, deltalon)
latgrid = np.arange(domain[2], domain[3] + .0001, deltalat)
if not os.path.exists(figdir):
    os.makedirs(figdir)

In [3]:
def read_oxy_woa(datafile, domain, filetype="woa"):
    with netCDF4.Dataset(datafile, "r") as nc:
        lon = nc.variables["lon"][:]
        lat = nc.variables["lat"][:]
        goodlon = np.where( (lon >= domain[0]) & (lon <= domain[1]))[0]
        goodlat = np.where( (lat >= domain[2]) & (lat <= domain[3]))[0]
        lon = lon[goodlon]
        lat = lat[goodlat]
        depth = nc.variables["depth"][:]
        
        if filetype == "woa":
            oxy = nc.variables["o_an"][0,:,goodlat,goodlon]
        elif filetype == "diva":
            oxy = nc.variables["Water body dissolved oxygen concentration"][:,goodlat,goodlon]
        
    return lon, lat, depth, oxy

In [ ]:
def plot_oxy_map(llon, llat, field, depth, figname=None):
    fig = plt.figure(figsize=(10, 10))
    ax = plt.subplot(111)
    pcm = m.pcolormesh(llon, llat, field, latlon=True, vmin=200., vmax=375.)
    cb = plt.colorbar(pcm, shrink=.6, extend="both")
    cb.set_label(r"$\mu$moles/kg", rotation=0, ha="left")
    cb.set_ticks(np.arange(200., 376., 25))
    m.drawmeridians(np.arange(-40., 60., 20.), zorder=2, labels=[0, 0, 0, 1], fontsize=14,  
                    linewidth=.25)
    m.drawparallels(np.arange(20., 80., 10.), zorder=2, labels=[1, 0, 0, 0], fontsize=14, 
                    linewidth=.25)
    m.fillcontinents(color=".85", zorder=3)
    m.drawcoastlines(linewidth=0.1, zorder=4)
    plt.title("Oxygen concentration at {} m".format(depth))
    if figname is not None:
        plt.savefig(figname, dpi=300, bbox_inches="tight")
    # plt.show()
    plt.close()

In [5]:
m = Basemap(projection='merc', llcrnrlon=domain[0], llcrnrlat=domain[2],
            urcrnrlon=domain[1], urcrnrlat=domain[3],
            lat_ts=0.5 * (domain[2] + domain[3]), resolution='i')

In [ ]:
lon1, lat1, depth1, oxy1 = read_oxy_woa(woadataurl, domain, filetype='woa')
lon2, lat2, depth2, oxy2 = read_oxy_woa(divafile, domain, filetype='diva')
llon1, llat1 = np.meshgrid(lon1, lat1)
llon2, llat2 = np.meshgrid(lon2, lat2)
depth2plot = 0

In [ ]:
plot_oxy_map(llon1, llat1, oxy1[0,:,:], depth2plot, os.path.join(figdir, "oxygen_WOA_V10.jpg"))
plot_oxy_map(llon2, llat2, oxy2[0,:,:], depth2plot, os.path.join(figdir, "oxygen_DIVAnd_V01.jpg"))

## Loop on the depths

In [ ]:
for ii, d1 in enumerate(depth1):
    #print("Working on depth {}".format(d2))
    #index1 = np.where(depth1 == d2)[0][0]

    
    plot_oxy_map(llon1, llat1, oxy1[ii,:,:], str(d1), 
                 os.path.join(figdir, "oxygen_WOA_year_{}.jpg".format(str(int(d1)).zfill(4))))